In [1]:
from mailbox import _PartialFile
import numpy as np
import xarray as xr
import os
import re
from cdo import *
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind
import skill_metrics as sm
from brokenaxes import brokenaxes

reload(ca)

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo()

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof
import statsmodels.api as sm
import metpy.constants as constants

# 模式数据年份合并及空间插值

In [ ]:
modelname = ["ACCESS-CM2", "BCC-CSM2-MR", "CAMS-CSM1-0", "CanESM5", "CESM2", "CESM2-WACCM", "CMCC-ESM2", "CNRM-CM6-1", "CNRM-ESM2-1", "EC-Earth3-Veg", "EC-Earth3", "FGOALS-g3", "GFDL-CM4", "HadGEM3-GC31-LL", "IITM-ESM", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", "KACE-1-0-G", "MIROC-ES2L", "MIROC6", "MPI-ESM1-2-HR", "MRI-ESM2-0", "NESM3", "NorESM2-LM", "TaiESM1", "UKESM1-0-LL"]
rlzn = ["r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r11i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f2", "r1i1p1f2", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f3", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f2", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r3i1p1f2"]
for model,rl in zip(modelname, rlzn):
    print(model)
    srcPath = "/home/ys17-23/Extension/CMIP6/CMIP6/"+ model +"/ssp245/" + rl + "/Amon"
    tmpPath = "/home/ys17-23/Extension/CMIP6/CMIP6/tmpPath"
    dstPath = "/home/ys17-23/Extension/personal-data/chenhj/CMIP6/ssp245"
    variable = ["hfls", "hus", "pr", "ps", "ta", "ts", "ua", "va", "wap", "zg"]
    freq = "Amon"
    ca.CMIP6_predealing_1(srcPath, tmpPath, dstPath, variable, freq, rl)

In [8]:
modelname = ["ACCESS-CM2", "BCC-CSM2-MR", "CAMS-CSM1-0", "CanESM5", "CESM2", "CESM2-WACCM", "CMCC-ESM2", "CNRM-CM6-1", "CNRM-ESM2-1", "EC-Earth3-Veg", "EC-Earth3", "FGOALS-g3", "GFDL-CM4", "HadGEM3-GC31-LL", "IITM-ESM", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", "KACE-1-0-G", "MIROC-ES2L", "MIROC6", "MPI-ESM1-2-HR", "MRI-ESM2-0", "NESM3", "NorESM2-LM", "TaiESM1", "UKESM1-0-LL"]
rlzn = ["r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r11i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f2", "r1i1p1f2", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f3", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f2", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r1i1p1f1", "r3i1p1f2"]
for model,rl in zip(modelname, rlzn):
    print(model)
    srcPath = "/home/ys17-23/Extension/CMIP6/CMIP6/"+ model +"/ssp245/" + rl + "/Amon"
    tmpPath = "/home/ys17-23/Extension/CMIP6/CMIP6/tmpPath"
    dstPath = "/home/ys17-23/Extension/personal-data/chenhj/CMIP6/ssp245"
    variable = ["tos"]
    freq = "Omon"
    ca.CMIP6_predealing_1(srcPath, tmpPath, dstPath, variable, freq, rl)

ACCESS-CM2
tos_Omon_ACCESS-CM2_ssp245_r1i1p1f1_gn_201501-210012.nc
BCC-CSM2-MR
tos_Omon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_201501-210012.nc
CAMS-CSM1-0
tos_Omon_CAMS-CSM1-0_ssp245_r1i1p1f1_gn_201501-209912.nc
CanESM5
tos_Omon_CanESM5_ssp245_r1i1p1f1_gn_201501-210012.nc
CESM2
tos_Omon_CESM2_ssp245_r11i1p1f1_gn_201501-206412.nc
tos_Omon_CESM2_ssp245_r11i1p1f1_gn_206501-210012.nc
CESM2-WACCM
tos_Omon_CESM2-WACCM_ssp245_r1i1p1f1_gn_201501-206412.nc
tos_Omon_CESM2-WACCM_ssp245_r1i1p1f1_gn_206501-210012.nc
CMCC-ESM2
tos_Omon_CMCC-ESM2_ssp245_r1i1p1f1_gn_201501-210012.nc
CNRM-CM6-1
tos_Omon_CNRM-CM6-1_ssp245_r1i1p1f2_gn_201501-210012.nc
CNRM-ESM2-1
tos_Omon_CNRM-ESM2-1_ssp245_r1i1p1f2_gn_201501-210012.nc
EC-Earth3-Veg
tos_Omon_EC-Earth3-Veg_ssp245_r1i1p1f1_gn_201501-201512.nc
tos_Omon_EC-Earth3-Veg_ssp245_r1i1p1f1_gn_201601-201612.nc
tos_Omon_EC-Earth3-Veg_ssp245_r1i1p1f1_gn_201701-201712.nc
tos_Omon_EC-Earth3-Veg_ssp245_r1i1p1f1_gn_201801-201812.nc
tos_Omon_EC-Earth3-Veg_ssp245_r1i1p1f1_gn_20190

In [3]:
modelname = ["ACCESS-CM2", "BCC-CSM2-MR", "CAMS-CSM1-0", "CanESM5", "CESM2", "CESM2-WACCM", "CMCC-ESM2", "CNRM-CM6-1", "CNRM-ESM2-1", "EC-Earth3-Veg", "EC-Earth3", "FGOALS-g3", "GFDL-CM4", "HadGEM3-GC31-LL", "IITM-ESM", "INM-CM4-8", "INM-CM5-0", "IPSL-CM6A-LR", "KACE-1-0-G", "MIROC-ES2L", "MIROC6", "MPI-ESM1-2-HR", "MRI-ESM2-0", "NESM3", "NorESM2-LM", "TaiESM1", "UKESM1-0-LL"]
yearstart = "201501"
yearend = ["210012", "210012", "209912", "209912", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012", "210012"]
dstPath = "/home/ys17-23/Extension/personal-data/chenhj/CMIP6/ssp245"
variable = ["hfls", "hus", "pr", "ps", "ta", "ts", "ua", "va", "wap", "zg"]

for var in variable:
    dstpath = os.path.join(dstPath, var)
    ca.CMIP6_check(dstpath, modelname, yearstart, yearend)

varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2099-12 time_num =  1020 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num =  1032 units =  W m-2
varname =  hfls 2015-01 2100-12 time_num

In [9]:
variable = ["hfls", "hus", "pr", "ps", "ta", "tos", "ts", "ua", "va", "wap", "zg"]
for var in variable:
    srcPath = "/home/ys17-23/Extension/personal-data/chenhj/CMIP6/ssp245/" + var
    dstPath = "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/" + var
    start = 2015
    end = 2099
    ca.p_year(srcPath, dstPath, start, end)

# 统一时间维度

In [12]:
path1 = "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp2452"
variable = ["hfls", "hus", "pr", "ps", "ta", "ts", "tos", "ua", "va", "wap", "zg"]
for var in variable:
    varpath = os.path.join(path1, var)
    g = os.walk(varpath)
    for path, dir_list, file_list in g:
        for filename in file_list:
            print(filename)
            ca.uniform_timestamp(os.path.join(path, filename), os.path.join("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245", var, filename), var, "20150101", "20991201", "MS")

hfls_Amon_CESM2-WACCM_ssp245_r1i1p1f1_gn_201501-209912.nc
hfls_Amon_MRI-ESM2-0_ssp245_r1i1p1f1_gn_201501-209912.nc
hfls_Amon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_201501-209912.nc
hfls_Amon_CAMS-CSM1-0_ssp245_r1i1p1f1_gn_201501-209912.nc
hfls_Amon_CESM2_ssp245_r11i1p1f1_gn_201501-209912.nc
hfls_Amon_CMCC-ESM2_ssp245_r1i1p1f1_gn_201501-209912.nc
hfls_Amon_CNRM-CM6-1_ssp245_r1i1p1f2_gr_201501-209912.nc
hfls_Amon_CNRM-ESM2-1_ssp245_r1i1p1f2_gr_201501-209912.nc
hfls_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_201501-209912.nc
hfls_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_201501-209912.nc
hfls_Amon_FGOALS-g3_ssp245_r1i1p1f1_gn_201501-209912.nc
hfls_Amon_GFDL-CM4_ssp245_r1i1p1f1_gr1_201501-209912.nc
hfls_Amon_HadGEM3-GC31-LL_ssp245_r1i1p1f3_gn_201501-209912.nc
hfls_Amon_INM-CM4-8_ssp245_r1i1p1f1_gr1_201501-209912.nc
hfls_Amon_INM-CM5-0_ssp245_r1i1p1f1_gr1_201501-209912.nc
hfls_Amon_KACE-1-0-G_ssp245_r1i1p1f1_gr_201501-209912.nc
hfls_Amon_MIROC-ES2L_ssp245_r1i1p1f2_gn_201501-209912.nc
hfls_Amon_MIROC6_ssp245_r1i

# 统一垂直维度

In [13]:
def uniform_plev(filepath, dstpath, var):
    plev = np.array([100000.0, 92500.0, 85000.0, 70000.0, 60000.0, 50000.0, 40000.0, 30000.0, 25000.0, 20000.0, 15000.0, 10000.0, 7000.0, 5000.0, 3000.0, 2000.0, 1000.0, 500.0, 100.0])
    f = xr.open_dataset(filepath)
    fvar = f[var]
    fvar.coords['plev'] = plev
    fvar.to_netcdf(dstpath)

path1 = "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp2452"
variable = ["zg", "ua", "va", "hus", "ta", "wap"]
for var in variable:
    varpath = os.path.join(path1, var)
    g = os.walk(varpath)
    for path, dir_list, file_list in g:
        for filename in file_list:
            print(filename)
            uniform_plev(os.path.join(path, filename), os.path.join("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245", var, filename), var)

zg_Amon_IPSL-CM6A-LR_ssp245_r1i1p1f1_gr_201501-209912.nc
zg_Amon_EC-Earth3-Veg_ssp245_r1i1p1f1_gr_201501-209912.nc
zg_Amon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_201501-209912.nc
zg_Amon_CAMS-CSM1-0_ssp245_r1i1p1f1_gn_201501-209912.nc
zg_Amon_CESM2_ssp245_r11i1p1f1_gn_201501-209912.nc
zg_Amon_CMCC-ESM2_ssp245_r1i1p1f1_gn_201501-209912.nc
zg_Amon_CNRM-CM6-1_ssp245_r1i1p1f2_gr_201501-209912.nc
zg_Amon_CNRM-ESM2-1_ssp245_r1i1p1f2_gr_201501-209912.nc
zg_Amon_EC-Earth3_ssp245_r1i1p1f1_gr_201501-209912.nc
zg_Amon_FGOALS-g3_ssp245_r1i1p1f1_gn_201501-209912.nc
zg_Amon_GFDL-CM4_ssp245_r1i1p1f1_gr1_201501-209912.nc
zg_Amon_HadGEM3-GC31-LL_ssp245_r1i1p1f3_gn_201501-209912.nc
zg_Amon_INM-CM4-8_ssp245_r1i1p1f1_gr1_201501-209912.nc
zg_Amon_INM-CM5-0_ssp245_r1i1p1f1_gr1_201501-209912.nc
zg_Amon_KACE-1-0-G_ssp245_r1i1p1f1_gr_201501-209912.nc
zg_Amon_MIROC-ES2L_ssp245_r1i1p1f2_gn_201501-209912.nc
zg_Amon_MIROC6_ssp245_r1i1p1f1_gn_201501-209912.nc
zg_Amon_MPI-ESM1-2-HR_ssp245_r1i1p1f1_gn_201501-209912.nc
zg_Amo

# 计算集合平均

In [14]:
def cdo_ensmean(srcPath, dstPath):
    g = os.walk(srcPath)
    inputString = ""
    mergelist = []
    for path, dir_list, file_list in g:
        for file_name in file_list:
            if re.search("IITM-ESM", file_name) == None and re.search("ensemble", file_name) == None:
                mergelist.append(os.path.join(path, file_name))
    for i in range(len(mergelist)):
        inputString += mergelist[i] + " "
    print(inputString)
    cdo.ensmean(input=inputString, output=os.path.join(dstPath, var + "_Amon_ensemble_ssp245_gn_201501-209912.nc"))
    
variables = ["hfls", "hus", "pr", "ps", "ta", "tos", "ts", "ua", "va", "wap", "zg"]
for var in variables:
    srcPath = os.path.join("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245", var)
    dstPath = os.path.join("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245", var)
    cdo_ensmean(srcPath, dstPath)

/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/hfls/hfls_Amon_CESM2-WACCM_ssp245_r1i1p1f1_gn_201501-209912.nc /home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/hfls/hfls_Amon_MRI-ESM2-0_ssp245_r1i1p1f1_gn_201501-209912.nc /home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/hfls/hfls_Amon_BCC-CSM2-MR_ssp245_r1i1p1f1_gn_201501-209912.nc /home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/hfls/hfls_Amon_CAMS-CSM1-0_ssp245_r1i1p1f1_gn_201501-209912.nc /home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/hfls/hfls_Amon_CESM2_ssp245_r11i1p1f1_gn_201501-209912.nc /home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/hfls/hfls_Amon_CMCC-ESM2_ssp245_r1i1p1f1_gn_201501-209912.nc /home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/hfls/hfls_Amon_CNRM-CM6-1_ssp245_r1i1p1f2_gr_201501-209912.nc /home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp245/

# 计算去趋势